# tutorial 1: basics

This tutorial shows you how to:
* build classifiers and regressors using the pipcecaster **MultichannelPipeline** class
* visualize multichannel pipeline architectures
* estimate pipeline accuracy using cross_val_score
* use pipecasater models for inference
* save & load models

## build pipelines with layering and broadcasting

The following cell contains code for building a fully functional classifier that takes 10 feature matrix inputs.

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pipecaster as pc

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(StandardScaler())
clf.add_layer(pc.MultichannelPredictor(KNeighborsClassifier()))

,layer_0,layer_1
channel,,
0,StandardScaler,KNeighborsClassifier_MC
1,StandardScaler,▽
2,StandardScaler,▽
3,StandardScaler,▽
4,StandardScaler,▽
5,StandardScaler,▽
6,StandardScaler,▽
7,StandardScaler,▽
8,StandardScaler,▽


**notes**

* Most pipecaster classes and functions can be accessed through the global pipecaster namespace, so you can import pipecaster as pc and access functionality with pc.FunctionalityA.
* Pipelines are displayed as pandas DataFrames when the pipeline name is on the last line of a Jupyter Notebook Cell.
* The DataFrame display shows input channel indices on the left and layer channel indices on the top.  These indices can be used to reference specific pipeline components.
* In line 5, a single StandardScaler intance is added to a layer with 10 channels.  The result is that the instance is broadcast automatically across all 10 channels.  
* The **MultichannelPredictor** class used in line 7 concatenates all input channels into a single feature matrix and feeds it as input to a single channel predictor, in this case KNeighborsClassifier.
* The **MultichannelPredictor** class used in line 7 is a single pipe instance, not a list of 10 pipe instances, so a broadcasting operation automatically occurs.  Because it is a multichannel pipeline component, recognized by the signature fit(Xs, y) rather than fit(X, y), the MultichannelPredictor is not duplicated 10 times during broadcasting as with the single channel component KNeighborsClassifier.  Instead, its input map is broadcast across all channels.  The result is a single MultichannelPredictor object with 10 inputs. 
* All pipecaster predictors must output a single prediction even though they may have multiple input matrices.  It would be difficult to interpret what to do with 10 different predictions for each sample.
* The MultichannelPredictor is listed as "KNeighborsClassifier_MC" in the display DataFrame.  The suffix stands for MultichannelPredictor.

### evaluate pipeline performance

In [21]:
Xs, y, matrix_types = pc.make_multi_input_classification(
    n_informative_Xs=10, n_random_Xs=0, n_weak_Xs=0)
accuracies = pc.cross_val_score(clf, Xs, y, cv=3)
print('Balanced accuracies for the 3 cross validation splits are: {}'.format(accuracies))

Balanced accuracies for the 3 cross validation splits are: [0.9705882352941176, 1.0, 0.9411764705882353]


**notes**

* Pipecaster provides synthetic data generators that make classification or regression datasets with multiple independent feature matrices.
* Pipecaster provides cross_val_score and cross_val_predict functions similar to those of scikit-learn, but the pipecaster versions take both multichannel and single channel predictors.
* Pipecaster's cross_val_score detects the type of predictor and uses balanced_accuracy_score for classifiers and explained_variance_score for regressors

### fit the pipeline

In [23]:
clf.fit(Xs, y)

,layer_0,out_0,layer_1,out_1
channel,,,,
0,StandardScaler,→,KNeighborsClassifier_MC,→
1,StandardScaler,→,▽,
2,StandardScaler,→,▽,
3,StandardScaler,→,▽,
4,StandardScaler,→,▽,
5,StandardScaler,→,▽,
6,StandardScaler,→,▽,
7,StandardScaler,→,▽,
8,StandardScaler,→,▽,


**notes**

* After model fitting, the display DataFrame now shows layer outputs as arrows.  This is useful for visualizing information flow when channel selectors inactivate some channels or when concatenators or predictors divert channel information.

### save and load the pipeline

In [27]:
clf.save('/tmp/pipeline_1.pkl')
clf2 = clf.load('/tmp/pipeline_1.pkl')
y_pred = clf2.predict(Xs)
for i in range(10):
    print('y_pred = {}, y_true = {}'.format(y_pred[i], y[i]))

y_pred = 0, y_true = 0
y_pred = 0, y_true = 0
y_pred = 1, y_true = 1
y_pred = 0, y_true = 0
y_pred = 0, y_true = 0
y_pred = 0, y_true = 0
y_pred = 0, y_true = 0
y_pred = 1, y_true = 1
y_pred = 1, y_true = 1
y_pred = 0, y_true = 0


## classifier/regressor dual function

The following cell contains code for building a fully functional regressor that takes 10 feature matrix inputs. Note that the same pipecaster classes are used as in example 1.1 (MultichannelPipeline, MultichannelPredictor).  Pipecaster classes automatically detect the estimator type and expose the proper value of the _estimator_type attribute and provide uniform label encoding/decoding for classifiers.

In [28]:
from sklearn.neighbors import KNeighborsRegressor

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(StandardScaler())
clf.add_layer(pc.MultichannelPredictor(KNeighborsRegressor()))

,layer_0,layer_1
channel,,
0,StandardScaler,KNeighborsRegressor_MC
1,StandardScaler,▽
2,StandardScaler,▽
3,StandardScaler,▽
4,StandardScaler,▽
5,StandardScaler,▽
6,StandardScaler,▽
7,StandardScaler,▽
8,StandardScaler,▽


## explicit channel concatenation

In example 1.1, MultichannelPredictor was used to concatenate and predict with multiple input channels.  Pipecaster also proves a **ChannelConcatenator** class which allows more explicit concatenation and prediction.

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pipecaster as pc

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(StandardScaler())
clf.add_layer(pc.ChannelConcatenator())
clf.add_layer(1, KNeighborsClassifier())

,layer_0,layer_1,layer_2
channel,,,
0,StandardScaler,ChannelConcatenator,KNeighborsClassifier
1,StandardScaler,▽,→
2,StandardScaler,▽,→
3,StandardScaler,▽,→
4,StandardScaler,▽,→
5,StandardScaler,▽,→
6,StandardScaler,▽,→
7,StandardScaler,▽,→
8,StandardScaler,▽,→


**notes**

* As with **MultichannelPredictor**, all 10 inputs are mapped to a single **ChannelConcatenator** component. ChannelConcatenator generates a single output into the first channel to which it has been mapped (channel 0 in this examples).
* In line 8, a new calling pattern for add_layers() is introduced.  This pattern is called channel mapping and consists of alternating integer / pipeline component parameters.  In this case, 1 input channel is mapped to KNeighborsClassifier (channel 0).  
* Arrow in layer_2 indicate pass-through channels to which no operation has been mapped.
* In this example,   
    `clf.add_layer(1, KNeighborsClassifier())`  
    could be replaced with   
    `clf.add_layer(KNeighborsClassifier())`.  
    Without channel mapping in the latter case, KNeighborsClassifier is broadcast across all 10 channels but 9 of the classifiers will not have inputs so they will never be instantiated or fit.
    

In [31]:
Xs, y, matrix_types = pc.make_multi_input_classification(
    n_informative_Xs=10, n_random_Xs=0, n_weak_Xs=0)
accuracies = pc.cross_val_score(clf, Xs, y, cv=3)
print('Balanced accuracies for the 3 cross validation splits are: {}'.format(accuracies))

Balanced accuracies for the 3 cross validation splits are: [1.0, 1.0, 0.96875]


## mapping components to channels

When we don't want to rely on broadcasting but need fine control over pipeline components, pipecaster provides two options: **component lists** and **channel mappings**.  Component list parameters specify one pipe per channel, and channnel mapping parameters specify enumerated blocks of channels.

The first example in this section illustrates the use of component lists, and the second illustrates the use of channel mappings.

In [37]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
import pipecaster as pc

clf = pc.MultichannelPipeline(n_channels=10)
component_list = [StandardScaler() for i in range(5)] + [MinMaxScaler() for i in range(5)]
clf.add_layer(component_list)
clf.add_layer(pc.MultichannelPredictor(KNeighborsClassifier()))

,layer_0,layer_1
channel,,
0,StandardScaler,KNeighborsClassifier_MC
1,StandardScaler,▽
2,StandardScaler,▽
3,StandardScaler,▽
4,StandardScaler,▽
5,MinMaxScaler,▽
6,MinMaxScaler,▽
7,MinMaxScaler,▽
8,MinMaxScaler,▽


**notes**

* StandardScalers have been mapped to the first five channels and MinMaxScalers have been mapped to the last five channels.

In [38]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
import pipecaster as pc

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(5, StandardScaler(), 5, MinMaxScaler())
clf.add_layer(pc.MultichannelPredictor(KNeighborsClassifier()))

,layer_0,layer_1
channel,,
0,StandardScaler,KNeighborsClassifier_MC
1,StandardScaler,▽
2,StandardScaler,▽
3,StandardScaler,▽
4,StandardScaler,▽
5,MinMaxScaler,▽
6,MinMaxScaler,▽
7,MinMaxScaler,▽
8,MinMaxScaler,▽


**notes**

* StandardScalers have been mapped to the first five channels and MinMaxScalers have been mapped to the last five channels.
* StandardScaler and MinMaxScaler are automatically broadcast (repeated) across the 5 channel blocks to which they are mapped.
* Pipecaster currently only allows a channel to map to one component.